**Проведите RFM анализ. В каждом подсегменте поделите пользователей на 4 класса. Отсчитывайте количество дней, прошедших с момента последней покупки, с максимальной даты покупки в датасете.**

1) Какое максимальное кол-во покупок было совершено одним пользователем?

2) Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)

3) Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?

4) Какая верхняя граница у количества покупок у пользователей с классом 2 в подсегменте R?

5) Сколько пользователей попало в сегмент 111?

6) Сколько пользователей попало в сегмент 311?

7) В каком RFM-сегменте самое большое кол-во пользователей?

8) В каком RFM-сегменте самое маленькое кол-во пользователей?

9) Какое количество пользователей попало в самый малочисленный сегмент?

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('RFM_ht_data.csv')

C:\Users\Gosha\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [5]:
df.shape

(332730, 4)

In [6]:
df.dtypes

InvoiceNo        object
CustomerCode     object
InvoiceDate      object
Amount          float64
dtype: object

In [7]:
# Приводим данные в нужный формат
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)
df.InvoiceNo = df.InvoiceNo.astype(str)
df.CustomerCode = df.CustomerCode.astype(str)

**Какое максимальное кол-во покупок было совершено одним пользователем?**

In [10]:
df.query('Amount > 0') \
    .groupby('CustomerCode', as_index=False) \
    .agg({'InvoiceNo': 'count'}) \
    .sort_values(by='InvoiceNo', ascending=False) \
    .rename(columns={'InvoiceNo': 'purchases'}) \
    .head()

,CustomerCode,purchases
89387,19057820,204
44593,13215452,113
10347,13032521,106
97076,19080880,99
119949,99003061,90


### Сегментация пользователей

In [11]:
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [12]:
# Дата последней покупки
last_date = df['InvoiceDate'].max()

In [13]:
rfm_table = (df.groupby('CustomerCode')
                .agg({'InvoiceDate': lambda x: (last_date - x.max()).days, #Recency #Кол-во дней с последней покупки
                      'InvoiceNo': lambda x: len(x),                       #Frequency #Количество заказов
                      'Amount': lambda x: x.sum()}))                       #Monetary Value #Общая сумма по всем заказам

rfm_table = rfm_table.rename(columns={'InvoiceDate': 'recency',
                                      'InvoiceNo': 'frequency',
                                      'Amount': 'monetary_value'})

# Приводит данные к нужному типу
rfm_table['recency'] = rfm_table['recency'].astype(int)
rfm_table['frequency'] = rfm_table['frequency'].astype(int)
rfm_table['monetary_value'] = rfm_table['monetary_value'].astype(float)

In [14]:
rfm_table.dtypes

recency             int32
frequency           int32
monetary_value    float64
dtype: object

In [15]:
rfm_table.head()

,recency,frequency,monetary_value
CustomerCode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88


In [16]:
rfm_segmentation = rfm_table

In [17]:
# Квантили для каждого столбца
quantiles_table = rfm_segmentation.quantile(q=[0.25, 0.5, 0.75])
quantiles_table

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [18]:
# функция для сегментации пользователей по давности покупки
def RClass(value, parameter_name, quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4

# функция для сегментации пользователей по частоте и сумме покупок
def FMClass(value, parameter_name, quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1


In [19]:
# сегментация по давности
rfm_segmentation['R_segm'] = rfm_segmentation['recency'] \
                                .apply(RClass, args=('recency', quantiles_table))

# сегментация по частоте
rfm_segmentation['F_segm'] = rfm_segmentation['frequency'] \
                                .apply(FMClass, args=('frequency', quantiles_table))

# сегментация по сумме покупок
rfm_segmentation['M_segm'] = rfm_segmentation['monetary_value'] \
                                .apply(FMClass, args=('monetary_value', quantiles_table))

# RFM для каждого пользователя
rfm_segmentation['RFM'] = rfm_segmentation['R_segm'].map(str) \
                            + rfm_segmentation['F_segm'].map(str) \
                            + rfm_segmentation['M_segm'].map(str)

In [20]:
rfm_segmentation.head()

,recency,frequency,monetary_value,R_segm,F_segm,M_segm,RFM
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443


**Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)**

In [23]:
quantiles_table.monetary_value.iloc[0]

765.0

**Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?**

In [24]:
rfm_segmentation.query('F_segm == 1').frequency.min()

4

**Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R?**

In [25]:
rfm_segmentation.query('R_segm == 2').recency.max()

8

**Сколько пользователей попало в сегмент 111?**

In [26]:
rfm_segmentation.query('RFM == "111"').shape[0]

9705

**Сколько пользователей попало в сегмент 311?**

In [27]:
rfm_segmentation.query('RFM == "311"').shape[0]

1609

**В каком RFM-сегменте самое большое кол-во пользователей?**

In [28]:
rfm_segmentation.value_counts('RFM').head(1)

RFM
444    10624
dtype: int64

**В каком RFM-сегменте самое маленькое кол-во пользователей?**

**Какое количество пользователей попало в самый малочисленный сегмент?**

In [29]:
rfm_segmentation.value_counts('RFM').tail(1)

RFM
414    2
dtype: int64